In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
base.reset_seed()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [5]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "trec"

In [6]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits_fine", f"~/data/{DATASET}/test-logits_fine", f"~/data/{DATASET}/train-logits-augmented_fine"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [8]:
vocab = base.get_vocab(all_data_tokens)

In [9]:
word_index = dict(zip(vocab, range(len(vocab))))

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [11]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

8766


In [12]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 8551 words (215) misses


In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [ ]:
num_epochs = 15
batch_size = 128

In [17]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [ ]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=50)

In [ ]:
base.reset_seed()

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [24]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base",
    n_trials=150
)

[I 2025-03-23 01:13:16,011] A new study created in memory with name: Base


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.458400,3.069126,0.177819,0.005790,0.020238,0.006485
2,2.834400,2.626046,0.363886,0.039890,0.074396,0.047460
3,2.468700,2.306453,0.439047,0.068129,0.100127,0.071554
4,2.196500,2.091045,0.471127,0.108610,0.118121,0.093770
5,2.009700,1.932238,0.527039,0.132902,0.151495,0.132010
6,1.830500,1.845821,0.532539,0.150639,0.157151,0.139438
7,1.711900,1.765708,0.549038,0.179995,0.169621,0.155691
8,1.629900,1.704964,0.571036,0.207771,0.192210,0.177835
9,1.536600,1.643923,0.575619,0.199444,0.187425,0.173288
10,1.456300,1.602813,0.593034,0.233051,0.208454,0.197623


[I 2025-03-23 01:14:15,358] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 0}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.919000,2.034140,0.484876,0.128003,0.136525,0.117001
2,1.678800,1.619525,0.604950,0.294426,0.245200,0.233528
3,1.174900,1.178951,0.704858,0.419537,0.361885,0.365682
4,0.733800,1.113917,0.715857,0.474927,0.426712,0.434513
5,0.433300,1.087022,0.751604,0.542359,0.508756,0.508896


[I 2025-03-23 04:21:06,245] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.002688402728586328, 'weight_decay': 0.002, 'warmup_steps': 2}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.778400,2.076880,0.478460,0.118378,0.131007,0.107522
2,1.744000,1.558154,0.612282,0.248966,0.247346,0.227710
3,1.225900,1.207063,0.696609,0.384152,0.351361,0.346501
4,0.833200,1.084556,0.732356,0.453426,0.435252,0.438903
5,0.552100,1.046944,0.741522,0.502998,0.460192,0.464932
6,0.367200,1.115579,0.736022,0.627105,0.517241,0.541263
7,0.221100,1.190699,0.760770,0.642092,0.570644,0.589537
8,0.136300,1.224378,0.755270,0.659573,0.598490,0.609030
9,0.072800,1.289222,0.770852,0.628477,0.594403,0.601562
10,0.038400,1.299283,0.766269,0.610677,0.606186,0.593995


[I 2025-03-23 04:22:03,252] Trial 149 pruned. 


In [25]:
print(best_trial)

BestRun(run_id='138', objective=0.6872468518180237, hyperparameters={'learning_rate': 0.004879497055330371, 'weight_decay': 0.002, 'warmup_steps': 2}, run_summary=None)


In [26]:
base.reset_seed()

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

In [31]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-destilace",
    n_trials=150
)

[I 2025-03-23 04:22:03,475] A new study created in memory with name: Test-destilace


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.234800,2.019891,0.176902,0.003538,0.020000,0.006012
2,1.862200,1.715170,0.368469,0.039467,0.073651,0.045954
3,1.635900,1.523278,0.432631,0.069515,0.096849,0.067902
4,1.472400,1.394872,0.476627,0.112013,0.119179,0.093869
5,1.362000,1.310333,0.512374,0.116109,0.136523,0.113681
6,1.258800,1.254398,0.523373,0.148252,0.149132,0.130490
7,1.199000,1.209889,0.541705,0.165743,0.162353,0.144619
8,1.149400,1.179047,0.574702,0.198799,0.188546,0.167292
9,1.100900,1.146723,0.571036,0.185979,0.179132,0.160836
10,1.057400,1.119623,0.590284,0.178956,0.200186,0.179331


[I 2025-03-23 04:22:55,045] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 4, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.785500,1.350547,0.483043,0.114514,0.130480,0.105334
2,1.142300,1.002925,0.637947,0.296287,0.251985,0.246978
3,0.818500,0.806351,0.725940,0.380529,0.357961,0.351950
4,0.596000,0.720394,0.746104,0.415648,0.406320,0.401757
5,0.434100,0.663186,0.765353,0.490501,0.446398,0.448692
6,0.323500,0.630522,0.768103,0.499129,0.466473,0.474692
7,0.232300,0.628278,0.779102,0.598041,0.519713,0.538725
8,0.191200,0.621611,0.780935,0.627792,0.566525,0.580954
9,0.143700,0.602461,0.789184,0.695254,0.618226,0.641116
10,0.119800,0.601583,0.791934,0.689965,0.626538,0.642534


[I 2025-03-23 07:23:10,041] Trial 148 finished with value: 0.6859120540876517 and parameters: {'learning_rate': 0.004220497038970213, 'weight_decay': 0.005, 'warmup_steps': 4, 'lambda_param': 0.4, 'temperature': 3.0}. Best is trial 129 with value: 0.710917878994125.


Trial 149 with params: {'learning_rate': 0.004665367038943749, 'weight_decay': 0.008, 'warmup_steps': 4, 'lambda_param': 0.5, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.797800,1.348902,0.497709,0.134315,0.137752,0.112982
2,1.139400,0.998105,0.647113,0.260062,0.271372,0.256358
3,0.810400,0.807651,0.711274,0.361899,0.351499,0.339765
4,0.581800,0.716683,0.745188,0.417631,0.401228,0.402811
5,0.412700,0.648010,0.768103,0.469001,0.454304,0.449133
6,0.298100,0.610265,0.780935,0.535784,0.484648,0.494614
7,0.220200,0.609309,0.788268,0.586897,0.528340,0.541163
8,0.171500,0.587382,0.799267,0.638343,0.600263,0.606458
9,0.130600,0.574505,0.800183,0.661103,0.643192,0.642234
10,0.109900,0.576421,0.800183,0.700864,0.655834,0.663658


[I 2025-03-23 07:24:27,512] Trial 149 finished with value: 0.6716909799449003 and parameters: {'learning_rate': 0.004665367038943749, 'weight_decay': 0.008, 'warmup_steps': 4, 'lambda_param': 0.5, 'temperature': 2.5}. Best is trial 129 with value: 0.710917878994125.


In [32]:
print(best_trial2)

BestRun(run_id='129', objective=0.710917878994125, hyperparameters={'learning_rate': 0.004953270489928273, 'weight_decay': 0.009000000000000001, 'warmup_steps': 4, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}, run_summary=None)


In [33]:
#Nápočet epoch na steps
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [34]:
base.reset_seed()

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-aug_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-aug_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [38]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

In [39]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base-aug",
    n_trials=150
)

[I 2025-03-23 07:24:27,870] A new study created in memory with name: Test-base-aug


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 39}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.791600,1.234699,0.679193,0.318515,0.308004,0.301556
2,0.705300,1.011577,0.725940,0.518921,0.431581,0.450676
3,0.365900,1.023958,0.758020,0.616590,0.525473,0.550786
4,0.198300,1.112690,0.764436,0.701392,0.604681,0.631250
5,0.116400,1.169958,0.771769,0.696173,0.620105,0.636465


[I 2025-03-23 07:25:43,076] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 8}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.757000,1.010443,0.807516,0.781515,0.700111,0.722714
2,0.056300,1.110456,0.825848,0.818385,0.728808,0.758874
3,0.022900,1.221233,0.816682,0.772324,0.703619,0.725131
4,0.017300,1.262079,0.813016,0.794622,0.724658,0.742528
5,0.009000,1.454345,0.818515,0.774300,0.720013,0.732202
6,0.006100,1.491798,0.819432,0.785125,0.716998,0.736722
7,0.007200,1.563513,0.809349,0.791529,0.710223,0.728185
8,0.006700,1.556836,0.812099,0.772482,0.702906,0.721896
9,0.003000,1.570837,0.816682,0.770857,0.719335,0.730453
10,0.001600,1.554241,0.821265,0.781651,0.717075,0.734840


[I 2025-03-23 16:39:44,836] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0039687155845044425, 'weight_decay': 0.005, 'warmup_steps': 49}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.700600,0.976059,0.817599,0.775128,0.698339,0.718287
2,0.050800,1.179364,0.821265,0.768623,0.719402,0.731652
3,0.024900,1.290448,0.813932,0.791752,0.736863,0.749726
4,0.015900,1.441395,0.818515,0.839140,0.718445,0.746368
5,0.011900,1.530180,0.817599,0.783817,0.702146,0.724443


[I 2025-03-23 16:41:41,808] Trial 149 pruned. 


In [40]:
print(best_trial3)

BestRun(run_id='107', objective=0.7879411361007308, hyperparameters={'learning_rate': 0.004876931770314574, 'weight_decay': 0.009000000000000001, 'warmup_steps': 49}, run_summary=None)


In [41]:
base.reset_seed()

In [42]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-aug_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-aug_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [43]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [44]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [45]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [46]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-Distill-aug",
    n_trials=150
)

[I 2025-03-23 16:41:42,184] A new study created in memory with name: Test-Distill-aug


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 39, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.226500,0.889369,0.675527,0.318360,0.275223,0.268805
2,0.595600,0.715353,0.747021,0.460645,0.410523,0.417359
3,0.395000,0.654787,0.767186,0.528512,0.463471,0.477897
4,0.285700,0.640508,0.776352,0.592815,0.504493,0.527473
5,0.218800,0.611250,0.799267,0.677510,0.589453,0.611789


[I 2025-03-23 16:43:34,822] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 46, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.534200,0.498172,0.833181,0.767483,0.659936,0.692283
2,0.110700,0.457325,0.847846,0.841221,0.733620,0.767663
3,0.082100,0.479839,0.837764,0.806066,0.724855,0.747748
4,0.075200,0.454842,0.857929,0.857505,0.755420,0.789894
5,0.071600,0.452270,0.853346,0.831655,0.753267,0.779244
6,0.068500,0.451903,0.857012,0.849252,0.760571,0.791227
7,0.068900,0.463028,0.847846,0.823569,0.743404,0.769123
8,0.067400,0.455583,0.851512,0.842261,0.744639,0.774648
9,0.065000,0.454302,0.851512,0.846649,0.745709,0.779661
10,0.063900,0.446010,0.857929,0.847025,0.751479,0.782446


[I 2025-03-24 03:47:28,906] Trial 148 finished with value: 0.7895455727268724 and parameters: {'learning_rate': 0.0042506356964603685, 'weight_decay': 0.002, 'warmup_steps': 38, 'lambda_param': 0.5, 'temperature': 2.0}. Best is trial 112 with value: 0.8063353096417676.


Trial 149 with params: {'learning_rate': 0.004107457138097337, 'weight_decay': 0.0, 'warmup_steps': 43, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.537600,0.489716,0.839597,0.766958,0.683255,0.707790
2,0.112100,0.477846,0.836847,0.823583,0.725228,0.756889
3,0.082300,0.463325,0.848763,0.833048,0.751780,0.775071
4,0.073100,0.466491,0.850596,0.824513,0.755560,0.775859
5,0.071900,0.471439,0.849679,0.830971,0.757666,0.778676
6,0.070200,0.466212,0.846929,0.828829,0.740987,0.764847
7,0.067800,0.489708,0.833181,0.817446,0.742846,0.765380
8,0.067100,0.464070,0.846929,0.831791,0.752522,0.777840
9,0.064700,0.467468,0.846013,0.831211,0.749077,0.775004
10,0.064000,0.477152,0.838680,0.814517,0.742685,0.763817


[I 2025-03-24 03:51:18,633] Trial 149 pruned. 


In [47]:
print(best_trial4)

BestRun(run_id='112', objective=0.8063353096417676, hyperparameters={'learning_rate': 0.004415119212034352, 'weight_decay': 0.001, 'warmup_steps': 42, 'lambda_param': 0.5, 'temperature': 2.0}, run_summary=None)
